## Chaining
We can **chain** transformations and action to create a computation **pipeline**
Suppose we want to compute the sum of the squares
$$ \sum_{i=1}^n x_i^2 $$
where the elements $x_i$ are stored in an RDD.

### Start the `SparkContext`

In [1]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

22/10/17 12:04:36 WARN Utils: Your hostname, pallavi-XPS-13-7390 resolves to a loopback address: 127.0.1.1; using 192.168.1.81 instead (on interface wlp2s0)
22/10/17 12:04:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/17 12:04:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/17 12:04:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/17 12:04:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
B=sc.parallelize(np.random.randint(0,10,size=1000))
lst = B.collect()
for i in lst: 
    print(i,end=', ')

7, 7, 5, 8, 3, 7, 3, 6, 1, 7, 5, 6, 5, 3, 1, 3, 4, 1, 6, 3, 6, 1, 8, 7, 8, 5, 6, 4, 8, 1, 8, 2, 7, 9, 5, 9, 7, 0, 4, 2, 0, 7, 8, 6, 6, 1, 0, 3, 9, 5, 6, 3, 1, 5, 9, 7, 9, 0, 2, 4, 3, 7, 7, 6, 1, 3, 5, 9, 8, 4, 0, 8, 8, 0, 2, 3, 3, 4, 6, 5, 1, 2, 5, 2, 2, 5, 3, 4, 6, 3, 0, 6, 0, 3, 1, 0, 6, 9, 2, 6, 0, 5, 3, 6, 3, 3, 3, 7, 9, 8, 6, 9, 6, 9, 0, 4, 1, 4, 1, 8, 5, 0, 4, 5, 4, 7, 8, 6, 4, 1, 2, 3, 9, 5, 7, 5, 9, 1, 7, 7, 1, 9, 7, 6, 3, 9, 2, 6, 8, 3, 7, 4, 1, 6, 7, 4, 1, 2, 1, 1, 6, 7, 9, 8, 7, 0, 0, 7, 9, 2, 3, 8, 6, 0, 1, 0, 4, 6, 3, 3, 7, 1, 9, 0, 8, 4, 5, 0, 5, 0, 0, 0, 7, 6, 2, 4, 4, 1, 3, 1, 5, 4, 8, 8, 8, 0, 7, 4, 9, 0, 4, 6, 1, 1, 7, 2, 9, 1, 4, 6, 2, 4, 7, 8, 0, 3, 7, 4, 1, 6, 1, 3, 0, 8, 0, 1, 6, 2, 1, 8, 5, 5, 0, 3, 8, 9, 2, 9, 5, 1, 7, 7, 6, 9, 0, 2, 0, 8, 3, 3, 1, 6, 8, 5, 2, 0, 0, 8, 9, 7, 5, 5, 4, 3, 9, 2, 7, 8, 7, 5, 8, 2, 4, 4, 3, 6, 1, 9, 6, 0, 8, 9, 3, 0, 2, 6, 9, 9, 8, 7, 1, 1, 6, 0, 5, 2, 4, 0, 7, 5, 9, 3, 0, 7, 1, 0, 6, 5, 3, 1, 2, 3, 7, 1, 6, 3, 0, 9, 5, 6, 7, 6, 6, 7

### Sequential syntax for chaining
Perform assignment after each computation

In [3]:
%%time
Squares=B.map(lambda x:x*x)
summation = Squares.reduce(lambda x,y:x+y)

CPU times: user 17.6 ms, sys: 8.89 ms, total: 26.5 ms
Wall time: 1.11 s


In [4]:
print(summation)

28724


### Cascaded syntax for chaining
Combine computations into a single cascaded command

In [5]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 16.1 ms, sys: 8.04 ms, total: 24.1 ms
Wall time: 251 ms


28724

### Both syntaxes mean exactly the same thing
The only difference:
* In the sequential syntax the intermediate RDD has a name `Squares`
* In the cascaded syntax the intermediate RDD is *anonymous*

The execution is identical!

### Sequential execution
The standard way that the map and reduce are executed is
* perform the map
* store the resulting RDD in memory
* perform the reduce

### Disadvantages of Sequential execution

1. Intermediate result (`Squares`) requires memory space.
2. Two scans of memory (of `B`, then of `Squares`) - double the cache-misses.

### Pipelined execution
Perform the whole computation in a single pass. For each element of **`B`**
1. Compute the square
2. Enter the square as input to the `reduce` operation.

### Advantages of Pipelined execution

1. Less memory required - intermediate result is not stored.
2. Faster - only one pass through the Input RDD.

In [6]:
sc.stop()